In [ ]:
!pip install grpcio
!pip install git+https://github.com/IINemo/isanlp
!pip install git+https://github.com/IINemo/isanlp_srl_framebank

In [ ]:
!docker run --rm -d -p 3334:3333 inemo/isanlp_udpipe

In [1]:
%load_ext autoreload
%autoreload 2
    
import sys
sys.path.append('../src')

In [2]:
import logging
logger = logging.getLogger('isanlp_srl_framebank')

import sys
logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))

logger.setLevel(logging.DEBUG)

In [3]:
from isanlp.processor_remote import ProcessorRemote

synt_proc = ProcessorRemote('192.168.1.69', 3334, 'default')

In [4]:
from isanlp_srl_framebank.processor_srl_framebank import ProcessorSrlFramebank


srl_parser = ProcessorSrlFramebank('../models/', 
                                   enable_model_for_unknown_predicates=True, 
                                   known_preds_embeddings_type='elmo',
                                   unknown_preds_embeddings_type='elmo')

Loading the model for known predicates...
Loading the model...
Done.
Loading embeddings...
Model has no embeddings! Loading http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-wiki_600k_steps.tar.gz


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
Using TensorFlow backend.


From /opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/deeppavlov/models/embedders/elmo_embedder.py:186: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

From /opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/deeppavlov/models/embedders/elmo_embedder.py:188: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

From /opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/deeppavlov/models/embedders/elmo_embedder.py:190: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

From /opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/deeppavlov/models/embedders/elmo_embedder.py:198: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.

Done.
Loading feature models...
Done.
Model for known predicates is loaded.
Loading the model...
Done.
Loading embeddings...
Model has no embeddings! Loading http://files.deeppavlov.ai/deeppavlo

In [5]:
from isanlp import PipelineCommon


ppl = PipelineCommon([
    (synt_proc, 
     ['text'], 
     {'tokens' : 'tokens', 
      'sentences' : 'sentences',
      'postag' : 'postag',
      'morph' : 'morph',
      'lemma' : 'lemma',
      'syntax_dep_tree' : 'syntax_dep_tree'}),
    (srl_parser,
     ['tokens', 'postag', 'morph', 'lemma', 'syntax_dep_tree'],
     {0 : 'srl'})
])

In [6]:
text = 'Мы поехали на дачу.'
ling_annots = ppl(text)

Using model for unknown predicates.
predicting
(1, 1024)
Done.
predicting
(1, 1024)
Done.
Solving linear sum task.
Finished.


In [7]:
def print_roles(lemma, role_annot):
    for sent_num, ann_sent in enumerate(role_annot):
        for event in ann_sent:
            print('=====Pred: {}'.format(lemma[sent_num][event.pred[0]]))
            for arg in event.args:
                print('Arg({}): {}'.format(arg.tag, lemma[sent_num][arg.begin]))

print_roles(ling_annots['lemma'], ling_annots['srl'])

=====Pred: поехали
Arg(субъект перемещения): мы
Arg(конечная точка): дачу
